In [19]:

import argparse

import time

from copy import deepcopy

from PIL import Image
import numpy as np
from tqdm import tqdm

import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms


try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC
import torchvision.models as models

import clip
from clip.custom_clip import get_coop
from clip.cocoop import get_cocoop
from data.imagnet_prompts import imagenet_classes
from data.datautils import AugMixAugmenter, build_dataset
from utils.tools import Summary, AverageMeter, ProgressMeter, load_model_weight, set_random_seed, create_logger
from data.cls_to_names import *
from data.fewshot_datasets import fewshot_datasets
from data.imagenet_variants import thousand_k_to_200, imagenet_a_mask, imagenet_r_mask, imagenet_v_mask
from clip_retrieval.clip_client import ClipClient, Modality
%load_ext autoreload
%autoreload 2

client = ClipClient(
    url="http://127.0.0.1:1234/knn-service",
    indice_name='laion_400m',
    modality=Modality.IMAGE,
    num_images=10,
    deduplicate=False,
)
client_backup = ClipClient(
    url="http://127.0.0.1:1234/knn-service",
    indice_name='laion_400m',
    modality=Modality.IMAGE,
    num_images=200,
    deduplicate=False,
)

client_backup2 = ClipClient(
    url="http://127.0.0.1:1234/knn-service",
    indice_name='laion_400m',
    modality=Modality.IMAGE,
    num_images=1000,
    deduplicate=False,
)

## Class to names mapping
fewshot_datasets = ['DTD', 'Flower102', 'Food101', 'Cars', 'SUN397', 
                    'Aircraft', 'Pets', 'Caltech101', 'UCF101', 'eurosat']
test_sets = 'Cars'
if test_sets == 'Caltech101':
    cls2names = ['face', 'leopard', 'motorbike', 'accordion', 'airplane', 'anchor', 'ant', 'barrel', 'bass', 'beaver', 'binocular', 'bonsai', 'brain', 'brontosaurus', 'buddha', 'butterfly', 'camera', 'cannon', 'car_side', 'ceiling_fan', 'cellphone', 'chair', 'chandelier', 'cougar_body', 'cougar_face', 'crab', 'crayfish', 'crocodile', 'crocodile_head', 'cup', 'dalmatian', 'dollar_bill', 'dolphin', 'dragonfly', 'electric_guitar', 'elephant', 'emu', 'euphonium', 'ewer', 'ferry', 'flamingo', 'flamingo_head', 'garfield', 'gerenuk', 'gramophone', 'grand_piano', 'hawksbill', 'headphone', 'hedgehog', 'helicopter', 'ibis', 'inline_skate', 'joshua_tree', 'kangaroo', 'ketch', 'lamp', 'laptop', 'llama', 'lobster', 'lotus', 'mandolin', 'mayfly', 'menorah', 'metronome', 'minaret', 'nautilus', 'octopus', 'okapi', 'pagoda', 'panda', 'pigeon', 'pizza', 'platypus', 'pyramid', 'revolver', 'rhino', 'rooster', 'saxophone', 'schooner', 'scissors', 'scorpion', 'sea_horse', 'snoopy', 'soccer_ball', 'stapler', 'starfish', 'stegosaurus', 'stop_sign', 'strawberry', 'sunflower', 'tick', 'trilobite', 'umbrella', 'watch', 'water_lilly', 'wheelchair', 'wild_cat', 'windsor_chair', 'wrench', 'yin_yang']
elif test_sets == 'DTD':
    cls2names = dtd_classes
elif test_sets =='Cars':
    cls2names = cars_classes
    # print(cls2names[:5])

In [20]:

def select_confident_samples(logits, top):
    batch_entropy = -(logits.softmax(1) * logits.log_softmax(1)).sum(1)
    idx = torch.argsort(batch_entropy, descending=False)[:int(batch_entropy.size()[0] * top)]
    return logits[idx], idx

def avg_entropy(outputs):
    # epsilon = 1e-10
    assert len(outputs) > 0
    assert torch.any(torch.isnan(outputs)) == False
    logits = outputs - outputs.logsumexp(dim=-1, keepdim=True) # logits = outputs.log_softmax(dim=1) [N, 1000]
    assert torch.any(torch.isnan(logits)) == False
    avg_logits = logits.logsumexp(dim=0) - np.log(logits.shape[0]) # avg_logits = logits.mean(0) [1, 1000]
    # print(avg_logits)
    if torch.any(torch.isnan(avg_logits)):
        print("average logits ", outputs.log_softmax(dim=1).mean(0))
    assert torch.any(torch.isnan(avg_logits)) == False
    
    min_real = torch.finfo(avg_logits.dtype).min
    avg_logits = torch.clamp(avg_logits, min=min_real)
    assert torch.any(torch.isnan(avg_logits)) == False
    return -((avg_logits) * (torch.exp(avg_logits))).sum(dim=-1)

In [21]:

def accuracy(output, target, topk=(1,), caption=None, logger=None):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)
        if output.shape[0] == 1:#only image prediction
            logit_k, pred = output.topk(maxk, 1, True, True)
            pred = pred.t()
            correct = pred.eq(target.view(1, -1).expand_as(pred))
        else: # evaluate captions
            bag = []
            # # length = max(5, output.shape[0]-1)
            # cap_pred = output[1:]
            # # cap_pred = torch.mean(cap_pred, 0,  keepdim=True)
            # _, pred = cap_pred.topk(maxk, 1, True, True) #5, 1 #candidate labels
            # pred = pred.reshape(maxk, 1)
            pred = torch.mean(output, 0, keepdim=True)
            _, pred = pred.topk(maxk, 1, True, True)
            pred = pred.reshape(maxk, 1)

            print("caption prediction" , pred)
            print("label ", target.view(1, -1).expand_as(pred))
            correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            if k == 1 and correct_k.item() == 0:
                print("-------wrong prediction-------")
                # logger.info("wrong prediction , logit: ", output)
                pred = pred.squeeze().tolist()
                pred = [cls2names[lb] for lb in pred]
                print("target: [{}]".format( cls2names[target.squeeze().item()]))
                print("predicted category & logit: {}".format(list(zip(pred, logit_k.squeeze().tolist()))))
                # print("logit ", logit_k)
                if logger: logger.info("wrong prediction, target {} & predicted value {}".format(target, pred))
                print("-------------------------------")
            elif k==1 and correct_k.item() == 1:
                print("-------correct prediction-------")
                # logger.info("wrong prediction , logit: ", output)
                pred = pred.squeeze().tolist()
                pred = [cls2names[lb] for lb in pred]
                print("target: [{}]".format( cls2names[target.squeeze().item()]))
                print("predicted category & logit: {}".format(list(zip(pred, logit_k.squeeze().tolist()))))
                print("-------------------------------")
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [22]:
#Parameters
tta_steps = 1
which_loss = "cosine"
gpu = 3
print_freq = 10

In [23]:

def test_time_tuning(model, inputs, optimizer, scaler, imagepath = None):
    # Entropy + Triplet loss function * 0.5
    # Triplet loss function, anchor = retrieved vocab, positive = top5, negative = bottom5
    for j in range(tta_steps):
        with torch.cuda.amp.autocast():
            output_img, text_features = model(inputs) # bs, n_cls, (1, 1000), logit/ n_cls, 512
            logit_scale = model.logit_scale.exp()
            loss = 0
            ent = avg_entropy(output_img)
            if which_loss in ["both", "cosine"]:
                print("Pretrained model Entropy: ", ent.item())
                retrieve_K = 4 #TODO: 1,2,4,8
                # retrieved_txt_prj, _ = model.nn_project(image=inputs, print_hits=False, topk=retrieve_K) # K, 512, retrieved text projection
                # assert len(retrieved_txt_prj) == retrieve_K, len(retrieved_txt_prj)
                try:
                    query_res = client.query(image=imagepath)[:retrieve_K]
                    retrieved_txt= [D['caption'] for D in query_res]
                    retrieved_url = [D['url'] for D in query_res]
                    retrieved_score = [D['similarity'] for D in query_res]
                except:
                    # print(client.query(image=imagepath))
                    query_res = client_backup.query(image=imagepath)[:retrieve_K]
                    retrieved_txt= [D['caption'] for D in query_res]
                    retrieved_url = [D['url'] for D in query_res]
                    retrieved_score = [D['similarity'] for D in query_res]
                if len(retrieved_txt) == retrieve_K:
                    caption_feat = model.forward_caption(retrieved_txt) #normalized, 
                    # output_merged = model.caption_ensemble(retrieved_txt, retrieved_score)
                else:
                    query_res = client_backup2.query(image=imagepath)[:retrieve_K]
                    retrieved_txt= [D['caption'] for D in query_res]
                    retrieved_url = [D['url'] for D in query_res]
                    retrieved_score = [D['similarity'] for D in query_res]
                    caption_feat = model.forward_caption(retrieved_txt)
                
                for i, c_i in enumerate(caption_feat):
                    print("Caption: ", retrieved_txt[i])
                    print("paired Image: ", retrieved_url[i])
                    c_i = c_i.reshape(1, -1)
                    ent = avg_entropy(logit_scale * c_i @ text_features.t())
                    print("caption entropy ", ent)
                
                # merged
                weighted =[]
                assert len(retrieved_score) == len(caption_feat), (len(retrieved_score), len(caption_feat))
                # print()
                for score, caption in zip(retrieved_score, caption_feat):
                    weighted.append(score/sum(retrieved_score) * caption)
                weighted = torch.mean(torch.stack(weighted), dim=0).reshape(1, -1)
                # assert weighted.shape == (1, img_feat.shape[-1]), weighted.shape
                criterion = torch.nn.CosineEmbeddingLoss()
                
                K=4
                top, top_ind = output_img.topk(2*K, 1, True, True)
                topK_ind = top_ind.squeeze()[:K]
                botK_ind = top_ind.squeeze()[K:]
                label = torch.tensor([1 for _ in range(K)] + [-1 for _ in range(K)]).cuda(gpu)
                pred = [cls2names[lb] for lb in topK_ind]
                # print(text_features[top_ind].shape, weighted.shape, label.shape)
                x1 = text_features[top_ind].squeeze()
                loss += criterion(x1, weighted.expand_as(x1), label)
                
                print("------before tuning --------")
                
                print("Prediced category & logit {}".format(list(zip(pred, top.squeeze()[:K]))))
                
                ## caption prediction
                caption_merged_logit = logit_scale * weighted @ text_features.t()
                top, top_ind = caption_merged_logit.topk(2*K, 1, True, True)
                topK_ind = top_ind.squeeze()[:K]
                botK_ind = top_ind.squeeze()[K:]
                pred = [cls2names[lb] for lb in topK_ind]
                print("Predicted category, logit of caption {}".format(list(zip(pred, top.squeeze()[:K]))))
                
                optimizer.zero_grad() 
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                
                # K = 5
                # top, top_ind = output_img.topk(2*K, 1, True, True)
                # # # print(top_ind.shape)
                # top5_ind = top_ind.squeeze()[:K]
                # bot5_ind = top_ind.squeeze()[K:]
                # assert len(top5_ind) == K and len(bot5_ind) == K, (top5_ind, bot5_ind)
                # # # bot5, bot5_ind = output_img.topk(K, 1, False, True)
                # # print("Top 5 ", top5_ind)
                # # # loss2 = avg_cos_dist(text_features[top5_ind], projections)
                # # x1 = torch.cat((text_features[top5_ind].squeeze(), text_features[bot5_ind].squeeze()), dim=0).cuda(gpu)
                # label = torch.tensor([1 for _ in range(K)] + [ -1 for _ in range(K)]).cuda(gpu)
                # loss2=[]
                # # for each in retrieved_txt_prj:
                # #     loss2.append(criterion(x1, each.expand_as(x1), label))
                # # print("Cosine embedding loss ", (sum(loss2)/len(loss2)).detach().cpu())
                # # loss += sum(loss2)/len(loss2)
            if which_loss in ["both", "entropy"]:
                logit_scale = model.logit_scale.exp()
                loss += avg_entropy(logit_scale * caption_feat @ text_features.t()) 
                print("Entropy loss: ", loss.detach().cpu())
                optimizer.zero_grad() 
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
    return retrieved_txt[0]

In [24]:
def test_time_adapt_eval(val_loader, model, model_state, optimizer, optim_state, scaler, save_result=False):
    batch_time = AverageMeter('Time', ':6.3f', Summary.NONE)
    top1 = AverageMeter('Acc@1', ':6.2f', Summary.AVERAGE)
    top2 = AverageMeter('Acc@2', ':6.2f', Summary.AVERAGE)
    top3 = AverageMeter('Acc@3', ':6.2f', Summary.AVERAGE)
    top4 = AverageMeter('Acc@4', ':6.2f', Summary.AVERAGE)
    top5 = AverageMeter('Acc@5', ':6.2f', Summary.AVERAGE)
    
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, top1, top2, top3, top4, top5],
        prefix='Test: ',
        logger = None)

    # reset model and switch to evaluate mode
    model.eval()
    with torch.no_grad():
        model.reset()
    end = time.time()
    cnt_empty = 0
    if save_result: Dict = dict.fromkeys(['image_path', 'caption', 'image_correct', 'caption_correct'])
    else: Dict=None
    
    for i, (image, target, imagepath) in tqdm(enumerate(val_loader)): 
        assert gpu is not None
        print("Image Path ", imagepath[0])
        
        target = target.cuda(gpu, non_blocking=True)
        
        ### One time training
        # reset the tunable prompt to its initial state

        with torch.no_grad():
            model.reset()
        optimizer.load_state_dict(optim_state)
                
        retrieved_caption = test_time_tuning(model, image.cuda(gpu, non_blocking=True), optimizer, scaler, imagepath[0])

        
        with torch.no_grad():
            with torch.cuda.amp.autocast():

                output, output_cap = model.inference(image.cuda(gpu, non_blocking=True), caption= None)

        acc1, acc2, acc3, acc4, acc5 = accuracy(output, target, topk=(1, 2, 3, 4, 5), caption=None, logger=None)
        # print(acc1, acc2, acc3, acc4, acc5)
        top1.update(acc1[0], image.size(0))
        top2.update(acc2[0], image.size(0))
        top3.update(acc3[0], image.size(0))
        top4.update(acc4[0], image.size(0))
        top5.update(acc5[0], image.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if (i+1) % print_freq == 0:
            progress.display(i)
        

    progress.display_summary()
    return [top1.avg, top2.avg, top3.avg, top4.avg, top5.avg], Dict

In [25]:
##parameters
fewshot_datasets = ['DTD', 'Flower102', 'Food101', 'Cars', 'SUN397', 
                    'Aircraft', 'Pets', 'Caltech101', 'UCF101', 'eurosat']
arch='ViT-B/16'
n_ctx=4
ctx_init="a_photo_of_a"
lr = 5e-3

In [26]:
# load model
if test_sets in fewshot_datasets:
    classnames = eval("{}_classes".format(test_sets.lower()))
model = get_coop(arch, test_sets, gpu, n_ctx, ctx_init)
model_state = None

cross_check = set()
for name, param in model.named_parameters():

    if "prompt_learner" not in name:
        param.requires_grad_(False)
    if param.requires_grad : cross_check.add(name)
print("tuing parameters ", cross_check)

print("=> Model created: visual backbone {}".format(arch))

assert gpu is not None
torch.cuda.set_device(gpu)
model = model.cuda(gpu)

trainable_param = model.prompt_learner.parameters()
optimizer = torch.optim.AdamW(trainable_param, lr)
optim_state = deepcopy(optimizer.state_dict())

# setup automatic mixed-precision (Amp) loss scaling
scaler = torch.cuda.amp.GradScaler(init_scale=1000)

print('=> Using native Torch AMP. Training in mixed precision.')

cudnn.benchmark = True

In [27]:
resolution = 224
workers = 4
dataset_mode = 'test'
data = '/data/seongha'
import sys
    # norm stats from clip.load()
normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                     std=[0.26862954, 0.26130258, 0.27577711])
    
    # iterating through eval datasets
datasets = test_sets.split("/")
results = {}

with open('tpt_classification_{}.txt'.format(test_sets), 'w') as f:
    sys.stdout = f
    for set_id in datasets:

        data_transform = transforms.Compose([
            transforms.Resize(resolution, interpolation=BICUBIC),
            transforms.CenterCrop(resolution),
            transforms.ToTensor(),
            normalize,
        ])
        batchsize = 1
        print("evaluating: {}".format(set_id))
        classnames = eval("{}_classes".format(set_id.lower()))
        model.reset_classnames(classnames, arch)

        val_dataset = build_dataset(set_id, data_transform, data, mode=dataset_mode)
        print("number of test samples: {}".format(len(val_dataset)))

        val_loader = torch.utils.data.DataLoader(
                    val_dataset,
                    batch_size=batchsize, shuffle=False,
                    num_workers=workers, pin_memory=True)
            
        results[set_id] = test_time_adapt_eval(val_loader, model, model_state, optimizer, optim_state, scaler)
        del val_dataset, val_loader

        try:
            print("=> Acc. on testset [{}]: @1 {}/ @2 {}/ @3 {}/ @4 {}/ @5 {}".format(set_id, results[set_id][0], results[set_id][1], results[set_id][2], results[set_id][3], results[set_id[4], results[set_id][5]]))
        except:
            print("=> Acc. on testset [{}]: {}".format(set_id, results[set_id]))
sys.stdout = sys.__stdout__

8041it [57:40,  2.32it/s]


In [ ]:
# num_val x (image path, caption, image_correct, caption_correct)
